In [1]:
# 변수의 shape, type, value 확인
def p(var,_str='') :
    if _str=='\n' or _str=='cr' :
        _str = '\n'
    else :
        print(f'<<{_str}>>:')
        _str = ''
    if type(var)!=type([]):
        try:
            print(f'Shape:{var.shape}')
        except :
            pass
    print(f'Type: {type(var)}')
    print(f'Values: {var}'+_str)

def pst(_x,_name=""):
    print(f'[{_name}] Shape{_x.shape}, {type(_x)}')
def ps(_x,_name=""):
    print(f'[{_name}] Shape{_x.shape}')

In [2]:
## Colab secrets에서 토큰 불러와서
## os environment에 저장
from google.colab import userdata
import os
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

!huggingface-cli whoami

garam827


## [evalute](https://huggingface.co/docs/evaluate/index) : NLP를 포함한 metric함수 lib. :  

In [3]:
%%capture
!pip install --upgrade datasets evaluate

### library install

In [4]:
import torch
import transformers

print(f"torch version:{torch.__version__}")
# 2505:2.6.0+cu124
print(f"transformers version:{transformers.__version__}")
# 2505:4.51.3

torch version:2.6.0+cu124
transformers version:4.52.4


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

# 1. Fine Tuning 전체 절차 한눈에 보기

## 1) Data준비, 2) Model 준비

In [6]:
# 데이터 준비: 'train'과 'test'로 나뉘어 있음, {train: Dataset({eatures: ['text', 'label'],num_rows: 25000}

dataset = load_dataset("imdb")#, cache_dir="/content/hf_cache"), download_mode="force_redownload")

# 모델 및 토크나이저 로드 : 'uncased', 소문자만 사용하는 작은 모델
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 시퀀스 분류를 위한 모델 로드 (2개의 클래스: 긍정/부정)
## 분류를 위한 새로운 head layer 자동으로 추가해 줌
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    device_map="auto")

# 데이터 전처리 : 토크나이징
def tokenize_function(examples): # input_ids, attention_mask, token_type_dis 반환
    # 'text' 컬럼을 토크나이징하고, 최대 길이로 패딩, 트렁케이션 적용
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# 데이터셋에 전처리 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 필요 없는 컬럼 제거 및 'label' 컬럼 이름 변경 (모델이 'labels'를 기대함)
# 학습에 사용될 'input_ids', 'attention_mask', 'token_type_ids', 'labels'만 남깁니다.
# 'token_type_ids'는 BERT 모델에 필요하지만, 다른 모델은 없을 수도 있습니다.
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# 데이터 형식을 PyTorch 텐서로 설정
tokenized_dataset.set_format("torch")

# 학습 및 평가 데이터셋 분리
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
p(dataset['train'][0])

<<>>:
Type: <class 'dict'>
Values: {'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex a

In [8]:
p(dataset,'dataset')
# Dataset({features: ['text', 'label'], num_rows: 25000})
print('=======')
p(tokenized_dataset,'tokenized_dataset')
# Dataset({features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'], num_rows: 25000 })
print('=======')
p(train_dataset[0],'train_dataset[0]')

<<dataset>>:
Shape:{'train': (25000, 2), 'test': (25000, 2), 'unsupervised': (50000, 2)}
Type: <class 'datasets.dataset_dict.DatasetDict'>
Values: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
<<tokenized_dataset>>:
Shape:{'train': (25000, 4), 'test': (25000, 4), 'unsupervised': (50000, 4)}
Type: <class 'datasets.dataset_dict.DatasetDict'>
Values: DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000


In [10]:
# --- 학습 시간 단축을 위해 데이터셋 일부만 사용 ---
p(tokenized_dataset,'tokenized_dataset')
# 학습 데이터셋 크기 설정
train_subset_size = 10000
# 평가 데이터셋 크기 설정
test_subset_size = 1000

# seed 값을 고정하면 매번 같은 무작위 샘플을 얻게 됩니다.
# 학습 데이터셋 서브셋 선택
train_dataset_subset = tokenized_dataset['train'].shuffle(seed=42).select(range(train_subset_size))
# 평가 데이터셋 서브셋 선택
test_dataset_subset = tokenized_dataset['test'].shuffle(seed=42).select(range(test_subset_size))

p(train_dataset_subset,'train_dataset_subset')
p(test_dataset_subset,'test_dataset_subset')

<<tokenized_dataset>>:
Shape:{'train': (25000, 4), 'test': (25000, 4), 'unsupervised': (50000, 4)}
Type: <class 'datasets.dataset_dict.DatasetDict'>
Values: DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
<<train_dataset_subset>>:
Shape:(10000, 4)
Type: <class 'datasets.arrow_dataset.Dataset'>
Values: Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})
<<test_dataset_subset>>:
Shape:(1000, 4)
Type: <class 'datasets.arrow_dataset.Dataset'>
Values: Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})


## 3) TrainingArguments 설정, 4) Trainer 정의

In [11]:
%%time
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./results",           # 학습 결과 및 체크포인트 저장 경로
    num_train_epochs=1,               # epochs 3->1
    per_device_train_batch_size=8,    # GPU/CPU당 batch seize <-- tuning
    per_device_eval_batch_size=8,     # GPU/CPU당 batch seize <-- tuning
    warmup_steps=100,                 # 웜업 스텝 수 500->100
    weight_decay=0.01,                # L2 정규화
    logging_dir="./logs",             # log 저장 경로
    logging_strategy="steps",         # 스텝 단위로 logging
    logging_steps=500,                # 500 스텝마다 logging
    eval_strategy="epoch",            # epoch 단위로 평가
    save_strategy="epoch",            # epoch 단위로 모델 저장
    load_best_model_at_end=True,      # 최상의 모델 로드
    metric_for_best_model="accuracy", # 최상 모델을 결정 지표
    report_to="tensorboard",          # TensorBoard로 로깅 보고 (설치 필요)
)

# metric 함수 정의 : accuracy
metric = evaluate.load("accuracy")    # HF lib.
def compute_metrics(eval_pred):       # Trainer에서 사용하기 위해
    logits, labels = eval_pred        # (pred, label)
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer 정의
trainer = Trainer(
    model=model,                      #
    args=training_args,               # 준비한 학습 조건
    train_dataset=train_dataset_subset,### 속도를 높이기 위해 일부만 사용
    eval_dataset=test_dataset_subset,  ### subset 사용
    tokenizer=tokenizer,              # DataCollatorWithPadding에서 사용
    compute_metrics=compute_metrics,  # 지표 계산 함수
) # DataCollatorWithPadding: batch단위로 padding하기, trainer에서 자동으로 작동됨

CPU times: user 162 ms, sys: 33.1 ms, total: 195 ms
Wall time: 1.67 s


<timed exec>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.


## 5) Training

In [12]:
%%time
# 학습 실행
trainer.train()
# Wall time: 7min 27s

Epoch,Training Loss,Validation Loss,Accuracy
1,0.296300,0.263551,0.915000


CPU times: user 7min 26s, sys: 2.19 s, total: 7min 28s
Wall time: 7min 31s


TrainOutput(global_step=1250, training_loss=0.3417175750732422, metrics={'train_runtime': 451.221, 'train_samples_per_second': 22.162, 'train_steps_per_second': 2.77, 'total_flos': 2631110553600000.0, 'train_loss': 0.3417175750732422, 'epoch': 1.0})

Epoch |	Training Loss | 	Validation Loss | 	Accuracy  
---|---|---|--  
1	| 0.326000 |	0.281118 |	0.913000

## 6) Evaluation,  7) Prediction

In [ ]:
%%time
# 평가 (학습 종료 후 최상 모델 로드됨)
eval_results = trainer.evaluate()
print(f"평가 결과: {eval_results}")

# 새로운 텍스트로 예측
text_to_predict = ["This movie was absolutely fantastic and I loved every minute of it!",
                   "This was a terrible waste of time, I regret watching it."]

# 토크나이징
inputs = tokenizer(
    text_to_predict,
    padding="max_length",
    truncation=True,
    return_tensors="pt" ).to(model.device)

# 예측 : trainer.predict(Dataset object) 또는 model(**inputs) 사용
with torch.no_grad():
    outputs = model(**inputs) # outputs는 logits를 포함
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1) # 예측된 레이블 ID

# 예측 결과 디코딩
sentiment_labels = ["negative", "positive"]
predicted_sentiments = [sentiment_labels[p.item()] for p in predictions]

print(f"\n예측 텍스트: {text_to_predict[0]}\n예측 감성: {predicted_sentiments[0]}")
print(f"예측 텍스트: {text_to_predict[1]}\n예측 감성: {predicted_sentiments[1]}")
# Wall time: 13.3 s

평가 결과: {'eval_loss': 0.25623250007629395, 'eval_accuracy': 0.916, 'eval_runtime': 13.3862, 'eval_samples_per_second': 74.704, 'eval_steps_per_second': 9.338, 'epoch': 1.0}

예측 텍스트: This movie was absolutely fantastic and I loved every minute of it!
예측 감성: positive
예측 텍스트: This was a terrible waste of time, I regret watching it.
예측 감성: negative
CPU times: user 13.5 s, sys: 31 ms, total: 13.5 s
Wall time: 13.4 s


## 7) Prediction-2 : trainer.predict() 사용하여 추론 하기  
#### Dataset Ojbect로 변환해서 넣어줘야 함: 전처리 추가됨

In [ ]:
%%time
from datasets import Dataset

text_to_predict = ["This movie was absolutely fantastic and I loved every minute of it!",
                   "This was a terrible waste of time, I regret watching it."]

inputs = tokenizer(text_to_predict, padding="max_length", truncation=True, return_tensors="pt")

# Dataset 객체로 변환 : 토크나이징된 데이터를 딕셔너리를 데이터셋으로 변환
predict_dataset = Dataset.from_dict({ # 예측이므로 'labels' 불필요
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
    'token_type_ids': inputs.get('token_type_ids') # BERT 모델의 경우 필요
})
predict_dataset.set_format("torch")

with torch.no_grad():
    outputs = trainer.predict(test_dataset=predict_dataset)
    # Trainer.predict()의 출력은 PredictionOutput 객체이며, 예측 결과는 .predictions에 있음
    logits = outputs.predictions
    predictions = np.argmax(logits, axis=-1) # NumPy 배열로 변환 후 argmax

# 예측 결과 디코딩 (이전 코드와 동일)
sentiment_labels = ["negative", "positive"]
predicted_sentiments = [sentiment_labels[p.item()] for p in predictions]

print(f"\n예측 텍스트: {text_to_predict[0]}\n예측 감성: {predicted_sentiments[0]}")
print(f"예측 텍스트: {text_to_predict[1]}\n예측 감성: {predicted_sentiments[1]}")
# Wall time: 39.9 ms


예측 텍스트: This movie was absolutely fantastic and I loved every minute of it!
예측 감성: positive
예측 텍스트: This was a terrible waste of time, I regret watching it.
예측 감성: negative
CPU times: user 40.1 ms, sys: 987 µs, total: 41.1 ms
Wall time: 39.9 ms


In [ ]:
p(inputs,'inputs')
p(predict_dataset,"predict_dataset")
p(outputs,'outputs')

<<inputs>>:
Type: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Values: {'input_ids': tensor([[ 101, 2023, 3185,  ...,    0,    0,    0],
        [ 101, 2023, 2001,  ...,    0,    0,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
<<predict_dataset>>:
Shape:(2, 3)
Type: <class 'datasets.arrow_dataset.Dataset'>
Values: Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids'],
    num_rows: 2
})
<<outputs>>:
Type: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Values: SequenceClassifierOutput(loss=None, logits=tensor([[-2.7093,  3.0032],
        [ 2.4921, -2.9399]], device='cuda:0'), hidden_states=None, attentions=None)


평가 결과: {'eval_loss': 0.28111812472343445, 'eval_accuracy': 0.913,  
      'eval_runtime': 14.9108, 'eval_samples_per_second': 67.065, 'eval_steps_per_second': 8.383, 'epoch': 1.0}  

예측 텍스트: This movie was absolutely fantastic and I loved every minute of it!  
예측 감성: positive  
예측 텍스트: This was a terrible waste of time, I regret watching it.  
예측 감성: negative  

## 추가 검토 사항

### 학습 가능한 최대 Batch size 찾기


In [ ]:
from accelerate import find_executable_batch_size

# 함수 호출 시 자동으로 배치 크기 탐색 함
@find_executable_batch_size(starting_batch_size=128)
def find_batch_size_and_build_trainer(batch_size):
    # 모델은 함수 내부에서 로드 (각 배치 크기 시도마다 새로 로드)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,)
            # device_map='auto') ## Accelerate가 memory handling에 제약 발생하므로 제거 함
    training_args = TrainingArguments(
        output_dir="./results_batch_size_search",
        num_train_epochs=1, # 배치 사이즈 탐색을 위해 1 에포크로 제한
        per_device_train_batch_size=batch_size, # 탐색된 배치 사이즈 적용
        per_device_eval_batch_size=batch_size,
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_dir="./logs_batch_size_search",
        logging_strategy="steps",
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="no",                   # 저장할 필요 없음
        load_best_model_at_end=False,         # 저장할 필요 없음
        max_steps=1,                          ####  1step 훈련 시도
        report_to="none",  )                  # repoert도 필요없음

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics, )

    trainer.train() # 1 스텝만 훈련 시도
    return batch_size  # 이 줄을 꼭 추가!

# find_batch_size_and_build_trainer 함수를 호출합니다.
# 데코레이터가 붙었으므로 이 호출이 배치 크기 탐색을 시작하고,
# 최종적으로 탐색된 배치 크기를 반환
executable_batch_size = find_batch_size_and_build_trainer()

print(f"최대 실행 가능한 배치 크기: {executable_batch_size}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-265342395d51>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-265342395d51>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


최대 실행 가능한 배치 크기: 64
